In [1]:
from pyspark.sql import SparkSession
import time

#Start stopwatch
startTime = time.time()

# (8 cores, 16gb per machine) x 5 = 40 cores

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.1.153:7077") \
        .appName("group_09")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","15s")\
        .config("spark.executor.cores",8)\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.NullPointerException
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:64)
	at org.apache.spark.storage.BlockManager.initialize(BlockManager.scala:248)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:510)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:423)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:238)
	at py4j.commands.Constructo

In [ ]:
import h5py
import sys
import io

rdd = spark_context.binaryFiles("hdfs://192.168.1.153:9000/millionsongs/data/*/*/*/*")

     # read a dataset and return it as a Python list #

def f(x):
    # return print(sys.path)
    
    # x[0] = filename
    # x[1] = binary content
    with h5py.File(io.BytesIO(x[1])) as f:
        # drill down with a path
        #return list(f.keys())
        g = f['analysis']['songs']['duration']
        # g is a 'dataset'
        return list(g)

rdd = rdd.map(f).cache()

In [ ]:
def time_conversion(duration):
    duration_in_half_minutes = duration[0]/30
    discretitized_duration = round(duration_in_half_minutes)
    discretized_duration_in_minutes = discretitized_duration/2
    return discretized_duration_in_minutes

converted_rdd = rdd.map(time_conversion)
print(converted_rdd.take(10))
print(converted_rdd.count())

In [ ]:
result = converted_rdd.map(lambda time: (time, 1)).reduceByKey(lambda a, b: a + b).sortBy(lambda x: x[0], True)
print(result.take(10))
endTime = time.time()
print("Time Elapsed {}".format((endTime-startTime)/60))

In [ ]:
import matplotlib.pyplot as plt
plt.bar(*zip(*result.collect()))
plt.show